In [30]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The forward and backward passes

### Get data

In [31]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from fastai.basics import *

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [32]:
path = datasets.download_data(MNIST_URL, ext='.gz'); path

PosixPath('/home/CW01/uia94835/.fastai/data/mnist.pkl.gz')

In [33]:
path = Config().data_path()/'mnist'
path

PosixPath('/home/CW01/uia94835/.fastai/data/mnist')

In [34]:
from exp.nb_01 import *

def normalize(x, m, s): return (x-m)/s

def get_data():
    with gzip.open(path/'mnist.pkl.gz', 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
        return map(tensor, (x_train,y_train,x_valid,y_valid))

In [35]:
x_train,y_train,x_valid,y_valid = get_data()

In [36]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [37]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [38]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-5.7450e-07), tensor(1.))

In [39]:
#export
def test_near_zero(a,tol=1e-3): assert np.abs(a)<tol, f"Near zero: {a}"

In [40]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [41]:
n,m = x_train.shape
c = (y_train.max()+1)
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [42]:
# num hidden
nh = 50

* HE init

In [43]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)
b1 = torch.randn(nh)
w2 = torch.randn(nh, 1) * math.sqrt(2/nh)
b2 = torch.randn(1)

In [44]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1*math.sqrt(2/m))

In [ ]:
def lin(inp, w, b):
    return inp @ w + b

In [ ]:
def relu(inp):
    return (inp>0).float()*inp

In [31]:
def mse(out, target):
    return (out.squeeze(-1)-target).pow(2).mean()

In [27]:
def model(inp, w1, b1, w2, b2):
    l1 = lin(inp, w1, b2)
    l1 = relu(l1)
    l2 = lin(l1, w2, b2)
    return l2

In [32]:
pred = model(x_train, w1, b2, w2, b2)
pred.shape

torch.Size([50000, 1])

In [34]:
loss = mse(pred, y_train)
print(loss)

tensor(28.4988)


### Gradients and backward pass

In [70]:
def mse_grad(inp, target):
    inp.g = 2. * (inp.squeeze()-target).unsqueeze(-1) / inp.shape[0]

In [71]:
def relu_grad(inp, out):
    inp.g = (inp>0).float() * out.g

In [78]:
def lin_grad(inp, w, b, out):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [79]:
def forward_and_backward(inp, targ):
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, w2, b2, out)
    relu_grad(l1, l2)
    lin_grad(inp, w1, b1, l1)

In [80]:
forward_and_backward(x_train, y_train)

In [81]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [82]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [83]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [84]:
loss = forward(xt2, y_train)

In [85]:
loss.backward()

In [86]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

### Refactor

In [125]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = (inp>0).float()*inp
        return self.out
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [126]:
class Lin():
    def __init__(self, w, b):
        self.w = w
        self.b = b
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b
        return self.out
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [127]:
class MSE():     
    def __call__(self, inp, target):
        self.inp = inp
        self.target = target
        self.out = (inp.squeeze(-1)-target).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze()-self.target).unsqueeze(-1) / self.inp.shape[0]

In [128]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = MSE()
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [129]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [135]:
%time loss = model(x_train, y_train)

CPU times: user 183 ms, sys: 140 ms, total: 323 ms
Wall time: 9.22 ms


In [45]:
%time model.backward()

NameError: name 'model' is not defined

### Refactor again 

In [98]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    def forward(self):
        raise NotImplementedError('not implemented!')
    def backward(self):
        self.bwd(self.out, *self.args)

In [99]:
class Relu(Module):
    def forward(self, inp): return (inp>0)*inp-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [131]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
    def forward(self, inp):
        return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = (inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [132]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze(-1)-targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2. * (inp.squeeze()-targ).unsqueeze(-1) / inp.shape[0]

In [133]:
class Model(Module):
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    def __call__(self, x, targ):
        for f in self.layers: x = f(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [134]:
w1.g,b1.g,w2.g,b2.g = [None]*4

In [135]:
model = Model()

In [136]:
%time loss = model(x_train, y_train)

CPU times: user 631 ms, sys: 19.4 ms, total: 650 ms
Wall time: 20 ms


In [137]:
%time model.backward()

CPU times: user 14.7 s, sys: 11.2 s, total: 25.9 s
Wall time: 787 ms
